In [1]:
# Imports

import sys
import os

# Add the upstream directory to sys.path
upstream_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if upstream_dir not in sys.path:
    sys.path.insert(0, upstream_dir)

# Now you can import the module
from opentrons_api import ot2_api
from microtissue_manipulator import core, utils
import numpy as np 
import cv2
import time
import json
import keyboard
# from pynput import keyboard
from configs import paths
import matplotlib.pyplot as plt
import requests
import datetime
import threading
import queue

In [2]:
# Turn on camera
calibration_profile = 'checkerboard'
cap = core.Camera(0, config_profile=calibration_profile, use_new_cam_mtx=True)

Using camera without buffer ...
Camera initialized ...


# Video test

In [61]:
window = cap.get_window()

robot_coords = []
camera_coords = []

while True:
    frame = cap.get_frame(undist=True)
    cv2.imshow(cap.window_name, frame)

    key_pressed = cv2.waitKey(1)

    if key_pressed == ord('q'):
        break

    elif key_pressed == ord('s'):
        timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H-%M-%S')
        filename = f"frame_{timestamp}.png"
        cv2.imwrite(os.path.join(paths.BASE_DIR, 'outputs', 'images', 'segmentation_test', filename), frame)
        print(f"Frame saved as {filename}")
   
cv2.destroyAllWindows()

# Init

In [3]:
# Connect the robot to the computer and this notebook
openapi = ot2_api.OpentronsAPI()

In [5]:
# Use the light control to see if the robot is connected as a sanity check
openapi.toggle_lights()

<Response [200]>

In [6]:
# Always do once after robot was just turned on
openapi.home_robot()

Request status:
<Response [200]>
{
  "message": "Homing robot."
}


<Response [200]>

In [6]:
# Use to restore labware and general run information after the notebook crashes
r = openapi.get_run_info()

Total number of runs: 20
Current run ID: 787c89c7-e7b5-4e02-a167-bdd1be3d5157
Current run status: idle


In [56]:
openapi.drop_tip_in_place()

<Response [201]>

In [7]:
# Do after first launch
openapi.create_run()

Request status:
<Response [201]>
{
  "data": {
    "id": "787c89c7-e7b5-4e02-a167-bdd1be3d5157",
    "ok": true,
    "createdAt": "2025-03-20T09:38:46.779778+00:00",
    "status": "idle",
    "current": true,
    "actions": [],
    "errors": [],
    "hasEverEnteredErrorRecovery": false,
    "pipettes": [],
    "modules": [],
    "labware": [],
    "liquids": [],
    "labwareOffsets": [],
    "runTimeParameters": [],
    "outputFileIds": []
  }
}


<Response [201]>

In [8]:
# Let the robot know that it has the P300 pipette
openapi.load_pipette()

Request status:
<Response [201]>
{
  "data": {
    "id": "894c9873-7303-4b3e-97cb-4e79570deefb",
    "createdAt": "2025-03-20T09:38:49.139000+00:00",
    "commandType": "loadPipette",
    "key": "894c9873-7303-4b3e-97cb-4e79570deefb",
    "status": "succeeded",
    "params": {
      "pipetteName": "p300_single_gen2",
      "mount": "left"
    },
    "result": {
      "pipetteId": "39f8aebd-bea2-4598-b614-a0381eb5ee51"
    },
    "startedAt": "2025-03-20T09:38:49.141386+00:00",
    "completedAt": "2025-03-20T09:38:51.171033+00:00",
    "intent": "setup",
    "notes": []
  }
}


<Response [201]>

# Labware declaration

### Opentrons 300 ul

In [9]:
#Define a tip rack. This is the default tip rack for the robot.
TIP_RACK = "opentrons_96_tiprack_300ul"
#Load the tip rack. Slot = 1 by default.
openapi.load_labware(TIP_RACK, 11)

Labware ID:
47352336-5da5-4b1d-ad33-1cb16cd4fcf9



<Response [201]>

### VWR 200 ul XL

In [10]:
custom_labware_path = os.path.join(paths.BASE_DIR,'protocols','vwr_96_tiprack_200ul_xl.json')
with open(custom_labware_path, 'r') as json_file:
    custom_labware = json.load(json_file)

command_dict = {
            "data": custom_labware
        }

command_payload = json.dumps(command_dict)

url = openapi.get_url('runs')+ f'/{openapi.run_id}/'+ 'labware_definitions'
r = requests.post(url = url, headers = openapi.HEADERS, params = {"waitUntilComplete": True}, data = command_payload)

In [11]:
#Define a tip rack. This is the default tip rack for the robot.
TIP_RACK = "vwr_96_tiprack_200ul_xl"
#Load the tip rack. Slot = 1 by default.
openapi.load_labware(TIP_RACK, 10, namespace='custom_beta',verbose=True)

Labware ID:
a343e5af-e587-4b4a-9128-7dfe465c937b



<Response [201]>

In [12]:
RESERVOIR = "corning_6_wellplate_16.8ml_flat"
openapi.load_labware(RESERVOIR, 3, namespace='opentrons',verbose=True)

Labware ID:
acb40fe1-acb9-4058-89c0-76476c0c34dd



<Response [201]>

In [13]:
# WELL_PLATE = "corning_96_wellplate_360ul_flat"
WELL_PLATE = "corning_384_wellplate_112ul_flat"
openapi.load_labware(WELL_PLATE, 6, namespace='opentrons',verbose=True)

Labware ID:
a4d63d5e-9940-408a-bfa4-c597a9acbcc5



<Response [201]>

In [14]:
r = openapi.pick_up_tip(openapi.labware_dct['10'], "A1")

In [48]:
r = openapi.pick_up_tip(openapi.labware_dct['11'], "A1")

In [27]:
openapi.drop_tip_in_place()

<Response [201]>

In [27]:
openapi.drop_tip(openapi.labware_dct['10'], "A1")

<Response [201]>

In [64]:
openapi.aspirate(openapi.labware_dct['9'], "A1", well_location='center', volume = 200, flow_rate = 200)

<Response [201]>

In [35]:
openapi.move_to_well(openapi.labware_dct['6'], "P24", well_location='top', offset=(0.9,-0.9,1))

<Response [201]>

# Filling a well plate

In [16]:
openapi.drop_tip_in_place()

<Response [201]>

In [50]:
# Take solution off the well palte with cuboids

columns = list(range(1,25))
# rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
rows = ['B']


row_idx = 0
column_idx = 0

while row_idx < len(rows):
    row = rows[row_idx]
    while column_idx < len(columns):
        column = columns[column_idx]
        r = openapi.aspirate(openapi.labware_dct['6'], f"{row}{column}", well_location = 'bottom', offset = (0.9,-0.9,0.5), volume = 50, flow_rate = 5)
        responce_dict = json.loads(r.text)['data']
        if responce_dict['status'] == 'failed':
            if responce_dict['error']['errorType'] == 'InvalidAspirateVolumeError':
                print('Dumping fluid')
                openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
        else:
            column_idx += 1
    column_idx = 0
    row_idx += 1

openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
openapi.move_relative('z', 20)


KeyboardInterrupt: 

In [7]:
openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)

<Response [201]>

In [9]:
r = openapi.dispense(openapi.labware_dct['3'], f"A1", well_location = 'top', volume = 10, flow_rate = 5)

In [51]:
openapi.move_relative('z', 20)

<Response [201]>

In [66]:
columns = list(range(1,25))
# rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
rows = ['G', 'H']

row_idx = 0
column_idx = 0

while row_idx < len(rows):
    row = rows[row_idx]
    while column_idx < len(columns):
        column = columns[column_idx]
        r = openapi.dispense(openapi.labware_dct['6'], f"{row}{column}", well_location = 'bottom', offset = (1,-1, 0), volume = 50, flow_rate = 200)
        responce_dict = json.loads(r.text)['data']
        if responce_dict['status'] == 'failed':
            if responce_dict['error']['errorType'] == 'InvalidDispenseVolumeError':
                print('Refilling pipette')
                openapi.aspirate(openapi.labware_dct['3'], "A1", well_location ='bottom', volume = 200, flow_rate = 200)
        else:
            column_idx += 1
    column_idx = 0
    row_idx += 1

openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
openapi.aspirate(openapi.labware_dct['3'], "A2", volume = 50, flow_rate = 200)
openapi.dispense(openapi.labware_dct['3'], "A2", volume = 50, flow_rate = 200)

Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette


<Response [201]>

In [27]:
openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
openapi.aspirate(openapi.labware_dct['3'], "A2", volume = 50, flow_rate = 200)
openapi.dispense(openapi.labware_dct['3'], "A2", volume = 50, flow_rate = 200)

<Response [201]>

In [ ]:
json.loads(r.text)['data']['error']['errorType']

{'id': 'dec84017-3246-49d5-8548-ccfe85aa2350',
 'createdAt': '2024-11-02T03:44:26.715454+00:00',
 'isDefined': False,
 'errorType': 'InvalidDispenseVolumeError',
 'errorCode': '4000',
 'detail': 'Cannot dispense 50.0 µL when only 0.0 µL has been aspirated.',
 'errorInfo': {},
 'wrappedErrors': []}

# Robot <-> camera calibration

In [62]:
squaresX=7
squaresY=5 
squareLength=0.022
markerLength=0.011
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
params = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, params)
board = cv2.aruco.CharucoBoard((squaresX, squaresY), squareLength, markerLength, aruco_dict)

manual_movement = utils.ManualRobotMovement(openapi)

openapi.move_to_coordinates((155,47,100), min_z_height=1, verbose=False)

window = cap.get_window()

robot_coords = []
camera_coords = []

while True:
 # Capture frame-by-frame
    frame = cap.get_frame(undist=True)

    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 100), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    center_screen_x = frame.shape[1] // 2
    center_screen_y = frame.shape[0] // 2
    cv2.circle(frame, (center_screen_x, center_screen_y), 5, (0, 0, 255), -1)
    cv2.putText(frame, f"Center: ({center_screen_x}, {center_screen_y})", (center_screen_x + 10, center_screen_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Calculate the center of each quarter of the screen
    quarter_centers = [
        (center_screen_x // 2, center_screen_y // 2),
        (3 * center_screen_x // 2, center_screen_y // 2),
        (center_screen_x // 2, 3 * center_screen_y // 2),
        (3 * center_screen_x // 2, 3 * center_screen_y // 2)
    ]

    # Draw circles at the center of each quarter
    for qx, qy in quarter_centers:
        cv2.circle(frame, (qx, qy), 5, (0, 255, 255), -1)
        cv2.putText(frame, f"({qx}, {qy})", (qx + 10, qy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    marker_corners, marker_ids, _ = detector.detectMarkers(frame)
    if marker_corners:
        for corner in marker_corners:
            corner = corner.reshape((4, 2))
            for point in corner:
                cv2.circle(frame, tuple(point.astype(int)), 5, (0, 255, 0), -1)

            center_x = int(np.mean(corner[:, 0]))
            center_y = int(np.mean(corner[:, 1]))
            cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
            cv2.putText(frame, f"({center_x}, {center_y})", (center_x + 10, center_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Calculate side lengths
    side_lengths = []
    if marker_corners:
        for corner in marker_corners[0]:
            for i in range(4):
                side_length = np.linalg.norm(corner[i] - corner[(i + 1) % 4])
                side_lengths.append(side_length)

    # Calculate the average side length
        average_side_length = np.mean(side_lengths)
        area = cv2.contourArea(marker_corners[0])
        one_d_ratio = 13.83 / average_side_length
        size_conversion_ratio = 13.83 ** 2 / area
        cv2.putText(frame, f"Area of marker: {area:.2f}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow(cap.window_name, frame)

    key_pressed = cv2.waitKey(1)

    if key_pressed == ord('q'):
        keyboard.unhook_all()
        break
    elif key_pressed == ord('s'):
        x, y, z = openapi.get_position(verbose=False)[0].values()
        robot_coords.append((x, y))
        camera_coords.append((center_x, center_y))

# When everything done, release the capture
# cap.release_camera()
cv2.destroyAllWindows()

calibration_data = utils.load_calibration_config(calibration_profile)
calibration_data['size_conversion_ratio'] = size_conversion_ratio
calibration_data['one_d_ratio'] = one_d_ratio
utils.save_calibration_config(calibration_profile, calibration_data)

In [63]:
calibration_points = [(160.0, 42.0),
 (150.0, 42.0),
 (150.0, 52.00000000000001),
 (160.0, 52.00000000000001)]

robot_coords = []
camera_coords = []


window = cap.get_window()

for calib_pt in calibration_points:
    openapi.move_to_coordinates((*calib_pt, 100), min_z_height=1, verbose=False)
    cv2.waitKey(1000)
    frame = cap.get_frame(undist=True)
    
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    center_screen_x = frame.shape[1] // 2
    center_screen_y = frame.shape[0] // 2
    cv2.circle(frame, (center_screen_x, center_screen_y), 5, (0, 0, 255), -1)
    cv2.putText(frame, f"Center: ({center_screen_x}, {center_screen_y})", (center_screen_x + 10, center_screen_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Calculate the center of each quarter of the screen
    quarter_centers = [
        (center_screen_x // 2, center_screen_y // 2),
        (3 * center_screen_x // 2, center_screen_y // 2),
        (center_screen_x // 2, 3 * center_screen_y // 2),
        (3 * center_screen_x // 2, 3 * center_screen_y // 2)
    ]

    # Draw circles at the center of each quarter
    for qx, qy in quarter_centers:
        cv2.circle(frame, (qx, qy), 5, (0, 255, 255), -1)
        cv2.putText(frame, f"({qx}, {qy})", (qx + 10, qy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    marker_corners, marker_ids, _ = detector.detectMarkers(frame)
    if marker_corners:
        for corner in marker_corners:
            corner = corner.reshape((4, 2))
            for point in corner:
                cv2.circle(frame, tuple(point.astype(int)), 5, (0, 255, 0), -1)

            center_x = int(np.mean(corner[:, 0]))
            center_y = int(np.mean(corner[:, 1]))
            cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
            cv2.putText(frame, f"({center_x}, {center_y})", (center_x + 10, center_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    cv2.waitKey(1)
    cv2.imshow(cap.window_name, frame)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    robot_coords.append((x, y))
    camera_coords.append((center_x, center_y))

cv2.destroyAllWindows()

# Write transformation matrix

In [64]:
calibration_data = utils.load_calibration_config(calibration_profile)

camera_coords = utils.sort_coordinates(camera_coords)
robot_coords = utils.sort_coordinates(robot_coords, reverse_y=True)

robot_to_camera_coords = {tuple(robot_coord): tuple(camera_coord) for robot_coord, camera_coord in zip(robot_coords, camera_coords)}
tf_mtx = utils.compute_tf_mtx(robot_to_camera_coords)

calibration_data['tf_mtx'] = tf_mtx.tolist()

utils.save_calibration_config(calibration_profile, calibration_data)

# Pipette offset calibration

### Blob detector

In [ ]:
calibration_data = utils.load_calibration_config(calibration_profile)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])


window = cap.get_window()

def on_mouse_click(event, x, y, flags, param):
    global circle_center
    global circle_radius
    global filtered_contours
    global X_init, Y_init

    if event == cv2.EVENT_MBUTTONDOWN:
        circle_center = (x, y)

    if event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:
            circle_radius += 10
        else:
            circle_radius -= 10

    if event == cv2.EVENT_LBUTTONDBLCLK:
        for contour in filtered_contours:
            r=cv2.pointPolygonTest(contour, (x,y), False)
            if r>0:
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)

                    x, y, _ = openapi.get_position(verbose=False)[0].values()
                    diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
                    X = X_init + diff[0] + offset[0]
                    Y = Y_init + diff[1] + offset[1]
                    
                    print(f"Robot coords: ({x}, {y})")
                    print(f"Clicked on: ({X}, {Y})")
                    openapi.move_to_coordinates((X, Y, 15), min_z_height=1, verbose=False)
                    # openapi.aspirate_in_place(flow_rate = 75, volume = 10)

                    
                else:
                    print("Contour center could not be found")

    if event == cv2.EVENT_RBUTTONDOWN:
        x, y, _ = openapi.get_position(verbose=False)[0].values()
        # openapi.move_to_coordinates((x, y, 100), min_z_height=1)
        openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=1, verbose=False)

        

cv2.setMouseCallback(cap.window_name, on_mouse_click)
circle_center = (int(1296.0), int(972.0))
circle_radius = 300
manual_movement = utils.ManualRobotMovement(openapi)

while True:
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    cv2.circle(frame, circle_center, circle_radius, (255, 0, 0), 2)
    # Create a mask with the same dimensions as the frame
    mask = np.zeros_like(frame, dtype=np.uint8)

    # Draw a filled circle on the mask
    cv2.circle(mask, circle_center, circle_radius, (255, 255, 255), -1)

    # Apply the mask to the frame
    masked_frame = cv2.bitwise_and(frame, mask)

    # Convert the masked frame to grayscale
    gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)
    # Apply thresholding to the grayscale image
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    # Fill the area outside the circle with black pixels
    # Convert the mask to grayscale
    mask_inv = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    thresh = cv2.bitwise_and(thresh, mask_inv)


    # Find contours in the masked frame
    contours, hei = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter the contours to exclude the outermost
    # filtered_contours = [contour for contour, h in zip(contours, hei[0]) if h[3] == 1]
    # Filter the contours by size
    filtered_contours = [contour for contour in contours if 15 < cv2.contourArea(contour) < 1000]
    # Draw the contours on the frame
    cv2.drawContours(frame, filtered_contours, -1, (0, 255, 0), 2)




    cv2.imshow(cap.window_name, frame)
    key_pressed = cv2.waitKey(1)
    if key_pressed == ord('o'):
        x, y, _ = openapi.get_position(verbose=False)[0].values()

        calibration_data['offset'] = [x-X_init, y-Y_init]

        utils.save_calibration_config(calibration_profile, calibration_data)

    elif key_pressed == ord('d'):
        openapi.dispense_in_place(flow_rate = 75, volume = 10)

    elif key_pressed == ord('a'):
        openapi.aspirate_in_place(flow_rate = 75, volume = 10)


    elif key_pressed == ord('q'):
        keyboard.unhook_all()
        break

        
cv2.destroyAllWindows()

### Crosshair detector

In [17]:
# Load the template image
path = os.path.join(paths.BASE_DIR, 'outputs', 'images', 'target_template.png')
template = cv2.imread(path, 0)  # Replace 'template.png' with your template image path
template_height, template_width = template.shape[:2]

calibration_data = utils.load_calibration_config(calibration_profile)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])

# Perform multi-object detection using template matching
# Start video stream
def on_mouse_click(event, x, y, flags, param):
    global X_init, Y_init
    if event == cv2.EVENT_LBUTTONDBLCLK:
        for (rect_x, rect_y, rect_w, rect_h) in rectangles:
            if rect_x <= x <= rect_x + rect_w and rect_y <= y <= rect_y + rect_h:
                cX = rect_x + rect_w // 2
                cY = rect_y + rect_h // 2

                X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)

                x, y, _ = openapi.get_position(verbose=False)[0].values()
                diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
                X = X_init + diff[0] + offset[0]
                Y = Y_init + diff[1] + offset[1]
                
                print(f"Robot coords: ({x}, {y})")
                print(f"Clicked on: ({X}, {Y})")
                openapi.move_to_coordinates((X, Y, 15), min_z_height=1, verbose=False)
                break

    if event == cv2.EVENT_RBUTTONDOWN:
        x, y, _ = openapi.get_position(verbose=False)[0].values()
        openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=1, verbose=False)

window = cap.get_window()
cv2.setMouseCallback(cap.window_name, on_mouse_click)
manual_movement = utils.ManualRobotMovement(openapi)


while True:
    # Capture frame-by-frame
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform template matching
    result = cv2.matchTemplate(gray_frame, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)


    # Eliminate overlapping matches
    locations = np.where(result >= 0.6)  # Adjust the threshold as needed
    rectangles = []
    for pt in zip(*locations[::-1]):
        rectangles.append([pt[0], pt[1], template_width, template_height])

    # Apply non-maximum suppression to remove overlapping rectangles
    rectangles, _ = cv2.groupRectangles(rectangles, groupThreshold=1, eps=0.2)
    for (x, y, w, h) in rectangles:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        center_x = x + w // 2
        center_y = y + h // 2
        cv2.circle(frame, (center_x, center_y), 3, (255, 0, 0), -1)
        cv2.putText(frame, f"({center_x}, {center_y})", (center_x + 10, center_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow(cap.window_name, frame)

    key_pressed = cv2.waitKey(1)
    # Break the loop on 'q' key press
    if key_pressed == ord('o'):
        x, y, _ = openapi.get_position(verbose=False)[0].values()

        calibration_data['offset'] = [x-X_init, y-Y_init]

        utils.save_calibration_config(calibration_profile, calibration_data)

    elif key_pressed == ord('d'):
        openapi.dispense_in_place(flow_rate = 75, volume = 10)

    elif key_pressed == ord('a'):
        openapi.aspirate_in_place(flow_rate = 75, volume = 10)


    elif key_pressed == ord('q'):
        keyboard.unhook_all()
        break


# Release resources
cv2.destroyAllWindows()

Robot coords: (155.0, 49.0)
Clicked on: (178.66306239951393, 128.24656073581696)
Robot coords: (155.0, 49.0)
Clicked on: (219.15437918272028, 128.47200658013935)
Robot coords: (155.0, 49.0)
Clicked on: (184.48212621976532, 142.68593711443174)
Robot coords: (155.0, 49.0)
Clicked on: (213.33489475455028, 114.05431487644134)
Robot coords: (155.0, 49.0)
Clicked on: (198.8285482484092, 141.93706808236743)


# Move to point

In [22]:
calibration_data = utils.load_calibration_config(calibration_profile)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])

def on_mouse_click(event, cX, cY, flags, param):
    global X_init, Y_init, X, Y, target_x, target_y
    if event == cv2.EVENT_LBUTTONDOWN:
        target_x, target_y = cX, cY
        # print("Clicked at pixel coordinate: ({}, {})".format(cX, cY))
        X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)

        x, y, _ = openapi.get_position(verbose=False)[0].values()
        diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
        X = X_init + diff[0] + offset[0]
        Y = Y_init + diff[1] + offset[1]
        # print(f"Robot coords: ({x}, {y})")
        # print(f"Clicked on: ({X}, {Y})")
        # openapi.move_to_coordinates((X, Y, 15), min_z_height=1)

    if event == cv2.EVENT_RBUTTONDOWN:
        x, y, _ = openapi.get_position(verbose=False)[0].values()
        openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=1, verbose=False)



# Create an instance of the ManualRobotMovement class
manual_movement = utils.ManualRobotMovement(openapi)

window = cap.get_window()
cv2.setMouseCallback(cap.window_name, on_mouse_click)

target_x, target_y = 0, 0

dish_bottom = 10# - 11.5
pickup_offset = 0.0 #0.6
flow_rate = 50
volume = 50

while True:
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 190), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Flow rate: {flow_rate} uL/s", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Volume: {volume} uL", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Pickup height: {dish_bottom + pickup_offset} mm", (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    center_screen_x = frame.shape[1] // 2
    center_screen_y = frame.shape[0] // 2
    # cv2.circle(frame, (center_screen_x, center_screen_y), 5, (0, 0, 255), -1)
    cv2.circle(frame, (target_x, target_y), 3, (0, 0, 255), -1)
    # cv2.putText(frame, f"Center: ({center_screen_x}, {center_screen_y})", (center_screen_x + 10, center_screen_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    cv2.imshow(cap.window_name, frame)
    # Draw a point at the center of the screen
    key_pressed = cv2.waitKey(1)

    if key_pressed == ord('q'):
        keyboard.unhook_all()
        break

    elif key_pressed == ord('o'):
        x, y, _ = openapi.get_position(verbose=False)[0].values()

        calibration_data['offset'] = [x-X_init, y-Y_init]

        utils.save_calibration_config(calibration_profile, calibration_data)

    elif key_pressed == ord('d'):
        openapi.dispense_in_place(flow_rate = flow_rate, volume = volume)

    elif key_pressed == ord('b'):
        openapi.blow_out_in_place(50)

    elif key_pressed == ord('a'):
        openapi.aspirate_in_place(flow_rate = flow_rate, volume = volume, verbose=True)

    elif key_pressed == ord('p'):
        openapi.move_to_coordinates((X, Y, dish_bottom+pickup_offset), min_z_height=dish_bottom)

    elif key_pressed == ord('s'):
        timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H-%M-%S')
        filename = f"frame_{timestamp}.png"
        cv2.imwrite(str(paths.BASE_DIR)+'\\outputs\\images\\2025-02-20_slice_test_coring\\'+filename, frame)
        print(f"Frame saved as {filename}")


cv2.destroyAllWindows()

Request status:
<Response [201]>
{
  "data": {
    "id": "94a6aaa4-2245-49e6-9be3-da14136c6e48",
    "createdAt": "2025-03-20T06:33:44.625393+00:00",
    "commandType": "moveToCoordinates",
    "key": "94a6aaa4-2245-49e6-9be3-da14136c6e48",
    "status": "succeeded",
    "params": {
      "minimumZHeight": 10.0,
      "forceDirect": false,
      "pipetteId": "27e99caa-0312-42c8-a6d2-442f8f36cfdd",
      "coordinates": {
        "x": 195.33259543126255,
        "y": 128.83952443264957,
        "z": 10.0
      }
    },
    "result": {
      "position": {
        "x": 195.33259543126255,
        "y": 128.83952443264957,
        "z": 10.0
      }
    },
    "startedAt": "2025-03-20T06:33:44.627465+00:00",
    "completedAt": "2025-03-20T06:33:46.241902+00:00",
    "intent": "setup",
    "notes": []
  }
}


In [13]:

for k in range(1):
    i = 0
    while i < 1:
        openapi.move_to_coordinates((X + i*3, Y - k*3, dish_bottom + pickup_offset), min_z_height=1)
        time.sleep(1)
        openapi.move_relative('z', -pickup_offset)
        time.sleep(1)
        r = openapi.aspirate_in_place(flow_rate = 200, volume = 100, verbose=True)
        responce_dict = json.loads(r.text)['data']

        for _ in range(3):
            openapi.move_relative('z', 0.2)
            time.sleep(0.2)
            openapi.move_relative('z', -0.2)
            time.sleep(0.2)

        if responce_dict['status'] == 'failed':
            if responce_dict['error']['errorType'] == 'InvalidAspirateVolumeError':
                print('Dumping fluid')
                openapi.move_relative('z', 50)
                openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='bottom', flow_rate = 200)
                openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='bottom', volume = 10, flow_rate = 200)
                openapi.dispense_in_place(flow_rate = 200, volume = 10)
                continue
        else:
            for j in range(24):
                openapi.move_relative('z', 0.25)
                time.sleep(0.3)
            i += 1
    openapi.move_relative('z', 50)
    openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='bottom', flow_rate = 200)
    openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='bottom', volume = 10, flow_rate = 200)
    openapi.dispense_in_place(flow_rate = 200, volume = 10)  
    openapi.move_relative('z', 50)
    

Request status:
<Response [201]>
{
  "data": {
    "id": "6e814e28-c9b5-4635-bc9e-4ce02771dc41",
    "createdAt": "2025-01-27T15:29:12.671515+00:00",
    "commandType": "moveToCoordinates",
    "key": "6e814e28-c9b5-4635-bc9e-4ce02771dc41",
    "status": "succeeded",
    "params": {
      "minimumZHeight": 1.0,
      "forceDirect": false,
      "pipetteId": "36cc037f-311f-49b0-a242-cccbb976be0f",
      "coordinates": {
        "x": 203.29450154025966,
        "y": 128.33746222135608,
        "z": 10.6
      }
    },
    "result": {
      "position": {
        "x": 203.29450154025966,
        "y": 128.33746222135608,
        "z": 10.6
      }
    },
    "startedAt": "2025-01-27T15:29:12.673555+00:00",
    "completedAt": "2025-01-27T15:29:14.880132+00:00",
    "intent": "setup",
    "notes": []
  }
}
Request status:
<Response [201]>
{
  "data": {
    "id": "2cac6c42-e6b3-4803-892f-98489eedb41f",
    "createdAt": "2025-01-27T15:29:16.984654+00:00",
    "commandType": "aspirateInPlace",
  

KeyboardInterrupt: 

In [19]:
openapi.move_relative('z', 10)

<Response [201]>

In [16]:
r = openapi.move_to_well(openapi.labware_dct['6'], 'A1', well_location='top', offset=(0,0,5), verbose=False, force_direct=True)

In [17]:
r = openapi.aspirate(openapi.labware_dct['6'], "A1", well_location ='bottom', volume = 10, flow_rate = 200)

In [27]:
json.loads(r.text)

{'data': {'id': '3ac6c403-d6b9-489c-8ac2-9ad7f0da571c',
  'createdAt': '2025-01-27T14:53:03.930897+00:00',
  'commandType': 'aspirate',
  'key': '3ac6c403-d6b9-489c-8ac2-9ad7f0da571c',
  'status': 'succeeded',
  'params': {'labwareId': 'c173eec9-1850-43cf-953e-6b4a1b081714',
   'wellName': 'A1',
   'wellLocation': {'origin': 'center',
    'offset': {'x': 0.0, 'y': 0.0, 'z': 0.0},
    'volumeOffset': 0.0},
   'flowRate': 200.0,
   'volume': 10.0,
   'pipetteId': '36cc037f-311f-49b0-a242-cccbb976be0f'},
  'result': {'position': {'x': 277.12, 'y': 166.99, 'z': 8.504999999999999},
   'volume': 10.0},
  'startedAt': '2025-01-27T14:53:03.933027+00:00',
  'completedAt': '2025-01-27T14:53:08.047486+00:00',
  'intent': 'setup',
  'notes': []}}

In [55]:
for k in range(1):
    i = 0
    while i < 1:
        openapi.move_to_coordinates((X + i*3, Y - k*3, dish_bottom + pickup_offset + 3 - 0.2), min_z_height=dish_bottom)
        time.sleep(1)
        for j in range(6):
            openapi.move_relative('z', -0.5)
            r = openapi.aspirate_in_place(flow_rate = flow_rate, volume = 10, verbose=True)
            time.sleep(0.3)

        # responce_dict = json.loads(r.text)['data']
        # if responce_dict['status'] == 'failed':
        #     if responce_dict['error']['errorType'] == 'InvalidAspirateVolumeError':
        #         print('Dumping fluid')
        #         openapi.move_relative('z', 50)
        #         openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
        #         openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='center', volume = 10, flow_rate = 200)
        #         openapi.dispense_in_place(flow_rate = 200, volume = 10)
        #         continue
        # else:
        #     for j in range(6):
        #         openapi.move_relative('z', 0.5)
        #         time.sleep(0.3)
            i += 1
    openapi.move_relative('z', 50)
    openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
    openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='center', volume = 10, flow_rate = 200)
    openapi.dispense_in_place(flow_rate = 200, volume = 10) 

Request status:
<Response [201]>
{
  "data": {
    "id": "5875e4d9-520c-4988-a692-3a5a81355796",
    "createdAt": "2025-01-26T16:33:03.561882+00:00",
    "commandType": "moveToCoordinates",
    "key": "5875e4d9-520c-4988-a692-3a5a81355796",
    "status": "succeeded",
    "params": {
      "minimumZHeight": 10.6,
      "forceDirect": false,
      "pipetteId": "9c7e4532-57e2-4cdf-bba7-261ce145309e",
      "coordinates": {
        "x": 201.1511350759473,
        "y": 124.81586979992255,
        "z": 13.8
      }
    },
    "result": {
      "position": {
        "x": 201.1511350759473,
        "y": 124.81586979992255,
        "z": 13.8
      }
    },
    "startedAt": "2025-01-26T16:33:03.564039+00:00",
    "completedAt": "2025-01-26T16:33:05.139211+00:00",
    "intent": "setup",
    "notes": []
  }
}
Request status:
<Response [201]>
{
  "data": {
    "id": "596eada9-f1be-457d-bfb4-aeb1eccfc469",
    "createdAt": "2025-01-26T16:33:06.337466+00:00",
    "commandType": "aspirateInPlace",
   

In [ ]:
openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='bottom', flow_rate = 200)
openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='bottom', volume = 10, flow_rate = 200)
openapi.dispense_in_place(flow_rate = 200, volume = 10)
openapi.move_relative('z', 50)

<Response [201]>

In [44]:
openapi.move_relative('z', 50)
openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='center', volume = 10, flow_rate = 200)
openapi.dispense_in_place(flow_rate = 200, volume = 10)

<Response [201]>

In [53]:
openapi.move_relative('z', 50)

<Response [201]>

In [23]:
openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 50)

<Response [201]>

In [24]:
openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='bottom', volume = 10, flow_rate = 200)

<Response [201]>

In [78]:
openapi.dispense(openapi.labware_dct['3'], "A1", well_location='bottom', volume = 200, flow_rate = 200)

<Response [201]>

# Hydrogel core detection

In [63]:
calibration_profile = 'standardDeck'
calibration_data = utils.load_calibration_config(calibration_profile)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])

def on_mouse_click(event, x, y, flags, param):
    global circle_center, circle_radius

    if event == cv2.EVENT_RBUTTONDOWN:
        x, y, _ = openapi.get_position(verbose=False)[0].values()
        openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=1, verbose=False)

    if event == cv2.EVENT_MBUTTONDOWN:
        circle_center = (x, y)

    if event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:
            circle_radius += 10
        else:
            circle_radius -= 10

# Create an instance of the ManualRobotMovement class
manual_movement = utils.ManualRobotMovement(openapi)

window = cap.get_window()
cv2.setMouseCallback(cap.window_name, on_mouse_click)

target_x, target_y = 0, 0

dish_bottom = 9.4# - 11.5
pickup_offset = 0 #0.6
flow_rate = 300
volume = 50

while True:
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 190), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Flow rate: {flow_rate} uL/s", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Volume: {volume} uL", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Pickup height: {dish_bottom + pickup_offset} mm", (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # masked_frame = cv2.bitwise_and(frame, mask)

    center_screen_x = frame.shape[1] // 2
    center_screen_y = frame.shape[0] // 2
    cv2.circle(frame, (target_x, target_y), 3, (0, 0, 255), -1)

    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(gray_img, dtype=np.uint8)
    cv2.circle(mask, circle_center, circle_radius, (255, 255, 255), -1)
    gray_img = cv2.GaussianBlur(gray_img, (21, 21), 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(gray_img)
    thresh_img = cv2.adaptiveThreshold(clahe_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 65, 9)
    kernel = np.ones((7, 7), np.uint8)
    opening = cv2.morphologyEx(thresh_img, cv2.MORPH_OPEN, kernel)
    masked_opening = cv2.bitwise_and(opening, mask)
    contours, _ = cv2.findContours(masked_opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    min_contour_area = 500
    max_contour_area = 5000

    circular_contours = []
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        area = cv2.contourArea(contour)
        if perimeter == 0 or area < min_contour_area or area > max_contour_area:
            continue
        circularity = 4 * np.pi * (area / (perimeter * perimeter))
        if 0.7 < circularity < 1.3:  # Adjust the range as needed
            circular_contours.append(contour)

    contours = circular_contours

    contour_centers = []
    for contour in contours:
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            contour_centers.append((cX, cY))
        else:
            contour_centers.append((0, 0))

    contour_img = frame.copy()
    cv2.circle(contour_img, circle_center, circle_radius, (255, 0, 0), 2)
    cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)
    # Draw contour centers on the image
    for center in contour_centers:
        cv2.circle(contour_img, center, 5, (255, 0, 0), -1)
    
    cv2.imshow(cap.window_name, contour_img)
    # Draw a point at the center of the screen
    key_pressed = cv2.waitKey(1)

    if key_pressed == ord('q'):
        keyboard.unhook_all()
        break

    elif key_pressed == ord('s'):
        timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H-%M-%S')
        filename = f"frame_{timestamp}.png"
        cv2.imwrite(str(paths.BASE_DIR)+'\\outputs\\images\\'+filename, frame)
        print(f"Frame saved as {filename}")


cv2.destroyAllWindows()

In [64]:
converted_contour_centers = []
x, y, _ = openapi.get_position(verbose=False)[0].values()
for center in contour_centers:
    cX, cY = center
    X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)
    diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
    X = X_init + diff[0] + offset[0]
    Y = Y_init + diff[1] + offset[1]
    converted_contour_centers.append((X, Y))

In [65]:
len(converted_contour_centers)

25

In [42]:
for i in range(10):
    x, y = converted_contour_centers[i]
    openapi.move_to_coordinates((x,y,12), min_z_height=1, verbose=False)
    time.sleep(0.1)

openapi.move_relative('z', 50)
    

# Picking procedure (semi-automatic)

In [67]:
cr = core.Core()

calibration_data = utils.load_calibration_config(calibration_profile)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])
size_conversion_ratio = calibration_data['size_conversion_ratio']
one_d_ratio = calibration_data['one_d_ratio']

vol = 10
dish_bottom = 10 #10.60 for 300ul, 9.5 for 200ul
pickup_offset = 0.5
pickup_height = dish_bottom + pickup_offset
flow_rate = 50
cuboid_size_theshold = (300, 450)
failure_threshold = 0.5
minimum_distance = 1.7

window = cap.get_window()

def create_well_mapping():
    rows = list("ABCDEFGHIJKLMNOP")
    columns = list(range(1, 25))
    well_mapping = {}

    for i in range(384):
        row = rows[i // 24]
        column = columns[i % 24]
        well_mapping[i] = f"{row}{column}"

    return well_mapping

well_mapping = create_well_mapping()

def on_mouse_click(event, x, y, flags, param):
    global circle_center
    global circle_radius
    global filtered_contours
    global X_init, Y_init

    if event == cv2.EVENT_MBUTTONDOWN:
        circle_center = (x, y)

    if event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:
            circle_radius += 10
        else:
            circle_radius -= 10

    if event == cv2.EVENT_LBUTTONDBLCLK:
        for contour in filtered_contours:
            r=cv2.pointPolygonTest(contour, (x,y), False)
            if r>0:
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)

                    x, y, _ = openapi.get_position(verbose=False)[0].values()
                    diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
                    X = X_init + diff[0] + offset[0]
                    Y = Y_init + diff[1] + offset[1]
                    
                    print(f"Robot coords: ({x}, {y})")
                    print(f"Clicked on: ({X}, {Y})")
                    openapi.move_to_coordinates((X, Y, 20), min_z_height=1, verbose=False)
                    time.sleep(0.1)
                    openapi.aspirate_in_place(flow_rate = flow_rate, volume = vol)
                    time.sleep(0.1)
                    openapi.move_to_coordinates((X, Y, pickup_height), min_z_height=1, verbose=False, force_direct=True)
                    time.sleep(1)
                    openapi.move_to_coordinates((X, Y, 30), min_z_height=1, verbose=False)
                else:
                    print("Contour center could not be found")

    if event == cv2.EVENT_RBUTTONDOWN:
        x, y, _ = openapi.get_position(verbose=False)[0].values()
        # openapi.move_to_coordinates((x, y, 100), min_z_height=1)
        openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=1, verbose=False)

        

cv2.setMouseCallback(cap.window_name, on_mouse_click)
circle_center = (int(1296.0), int(972.0))
circle_radius = 900
manual_movement = utils.ManualRobotMovement(openapi)

# Which wells to fill
columns = list(range(1,25))
#rows = ['F', 'G', 'H', 'I']
rows = ['G']

wells_to_fill = [f"{row}{column}" for row in rows for column in columns]

reversed_well_mapping = {v: k for k, v in well_mapping.items()}
wells_to_fill_indices = [reversed_well_mapping[well] for well in wells_to_fill]

idx = wells_to_fill_indices[0]
end_idx = wells_to_fill_indices[-1] + 1

cuboid_chosen = False
cuboid_choice = None
times = []
while idx < end_idx:
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.circle(frame, circle_center, circle_radius + int(minimum_distance / one_d_ratio), (0, 0, 255), 2)
    mask = np.zeros_like(frame, dtype=np.uint8)
    cv2.circle(mask, circle_center, circle_radius + int(minimum_distance / one_d_ratio), (255, 255, 255), -1)
    masked_frame = cv2.bitwise_and(frame, mask)
    cv2.circle(frame, circle_center, circle_radius, (0, 255, 0), 2)

    gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (11, 11), 0)
    thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,25,2) 
    kernel = np.ones((3,3),np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    mask_inv = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    thresh = cv2.bitwise_and(thresh, mask_inv)

    # Find contours in the masked frame
    contours, hei = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cr.cuboids = contours
    filtered_contours = [contour for contour in cr.cuboids if 30 < cv2.contourArea(contour) < 700]
    cr.cuboid_dataframe(filtered_contours)

    cuboid_size_micron2 = cr.cuboid_df.area * size_conversion_ratio * 1000000
    cuboid_diameter = 2 * np.sqrt(cuboid_size_micron2 / np.pi)
    dist_mm = cr.cuboid_df.min_dist * one_d_ratio
    cr.cuboid_df['diameter_microns'] = cuboid_diameter
    cr.cuboid_df['min_dist_mm'] = dist_mm
    
    # Filter out elongated contours

    pickable_cuboids = cr.cuboid_df.loc[(cuboid_size_theshold[0] < cr.cuboid_df['diameter_microns']) & 
                                        (cr.cuboid_df['diameter_microns'] < cuboid_size_theshold[1]) &
                                        ((cr.cuboid_df['aspect_ratio'] > 0.75) | (cr.cuboid_df['aspect_ratio'] < 1.25)) &
                                        (cr.cuboid_df['circularity'] > 0.6)].copy()

    # Check if cuboid centers are within the circle radius from the current circle center
    pickable_cuboids['distance_to_center'] = pickable_cuboids.apply(
        lambda row: np.sqrt((row['cX'] - circle_center[0])**2 + (row['cY'] - circle_center[1])**2), axis=1
    )
    pickable_cuboids = pickable_cuboids[pickable_cuboids['distance_to_center'] <= circle_radius]
    isolated = pickable_cuboids.loc[pickable_cuboids.min_dist_mm > minimum_distance]
    draw = isolated.contour.values.tolist()
    cv2.drawContours(frame, filtered_contours, -1, (0, 0, 255), 2)
    cv2.drawContours(frame, pickable_cuboids.contour.values.tolist(), -1, (0, 255, 255), 2)
    cv2.drawContours(frame, draw, -1, (0, 255, 0), 2)

    if not cuboid_chosen and len(isolated) > 0:
        if cuboid_choice is not None:
            prev_x, prev_y = cuboid_choice[['cX', 'cY']].values[0]
            
            cv2.circle(frame, (int(prev_x), int(prev_y)), int(round(failure_threshold / one_d_ratio)), (255, 0, 0), 2)
            distances = cr.cuboid_df.apply(lambda row: np.sqrt((row['cX'] - prev_x)**2 + (row['cY'] - prev_y)**2), axis=1).to_numpy()
            distances *= one_d_ratio
            if any(distances <= failure_threshold):
                print("Miss detected ...")
                idx -= 1

        cuboid_choice = isolated.sample(n=1)
        cuboid_chosen = True
        # cv2.imwrite(str(paths.BASE_DIR)+'\\outputs\\images\\'+f"frame_{idx}.png", frame)
        

    # for i, row in cr.cuboid_df.iterrows():
    #     cX, cY = int(row['cX']), int(row['cY'])
    #     aspect_ratio = row['aspect_ratio']
    #     circularity = row['circularity']
    #     cv2.putText(frame, f"{aspect_ratio:.2f}, {circularity:.2f}", (cX+20, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

    if cuboid_chosen:    
        cv2.drawContours(frame, cuboid_choice.contour.values.tolist(), -1, (255, 0, 0), 2)

    cv2.imshow(cap.window_name, frame)

    #-----------------------------------------------INTERACTION------------------------------------------------
    key_pressed = cv2.waitKey(1)

    if key_pressed == ord('q'):
        break

    elif key_pressed == ord('m'):
        start_time = time.time()
        print(len(isolated))
        if len(isolated) > 2:
            chosen = isolated.sample(n=2)
        else:
            chosen = isolated.copy()

        x, y, _ = openapi.get_position(verbose=False)[0].values()
        for i in range(len(chosen)):
            cX, cY = chosen.iloc[i][['cX', 'cY']].values

            X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)

            diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
            X = X_init + diff[0] + offset[0]
            Y = Y_init + diff[1] + offset[1]

            openapi.move_to_coordinates((X, Y, pickup_height+20), min_z_height=dish_bottom, verbose=False, force_direct=True)
            openapi.move_to_coordinates((X, Y, pickup_height), min_z_height=dish_bottom, verbose=False, force_direct=True)
            openapi.aspirate_in_place(flow_rate = flow_rate, volume = 10)
            openapi.move_relative('z', 10)
            openapi.aspirate_in_place(flow_rate = flow_rate, volume = 20)
            time.sleep(0.1)
        openapi.move_relative('z', 20)

        # wells = ['B11', 'B12', 'B13', 'B14', 'B15']
        openapi.move_to_well(openapi.labware_dct['6'], well_mapping[idx], well_location='top', offset=(0.9,-0.9,5), verbose=False, force_direct=True)
        for j in range(len(chosen)):
            if j == 0:
                air_disp_vol = 25
                liq_disp_vol = 10
            elif j == 1:
                air_disp_vol = 15
                liq_disp_vol = 10

            openapi.dispense(openapi.labware_dct['6'], well_mapping[idx], well_location='top', offset = (0.9,-0.9,5), volume = air_disp_vol, flow_rate = 50)
            time.sleep(0.1)
            openapi.dispense(openapi.labware_dct['6'], well_mapping[idx], well_location='bottom', offset = (0.9,-0.9,0), volume = liq_disp_vol, flow_rate = 50)
            time.sleep(0.3)
            idx += 1

        openapi.move_relative('z', 20)
        openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=1, verbose=False, force_direct=True)
        end_time = time.time()
        print(f"Time taken: {end_time - start_time:.2f} seconds")
        times.append(end_time - start_time)

    # elif key_pressed == ord('n'):
    #     for i in range(2):
    #         # if i == 0:
    #         #     air_disp_vol = 30
    #         #     liq_disp_vol = 10
    #         # elif i == 4:
    #         #     air_disp_vol = 10
    #         #     liq_disp_vol = 10
    #         # else:
    #         #     air_disp_vol = 20
    #         #     liq_disp_vol = 10
    #         # openapi.move_relative('z', 10)
    #         openapi.dispense_in_place(volume = 20, flow_rate = 50)
    #         time.sleep(1)
    #         openapi.move_relative('z', -10)
    #         openapi.dispense_in_place(volume = 10, flow_rate = 50)
    #         time.sleep(1)
    #         openapi.move_relative('z', 10)
    #         openapi.move_relative('x', -8.5)

        

    elif key_pressed == ord('c'):
        if len(isolated) > 0:
            cuboid_choice = isolated.sample(n=1)

    elif key_pressed == ord('p'):
        if len(isolated) == 0:
            print("No cuboids found in the selected region")
            continue

        cX, cY = cuboid_choice[['cX', 'cY']].values[0]

        X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)

        x, y, _ = openapi.get_position(verbose=False)[0].values()
        diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
        X = X_init + diff[0] + offset[0]
        Y = Y_init + diff[1] + offset[1]
        
        openapi.move_to_coordinates((X, Y, pickup_height), min_z_height=dish_bottom, verbose=False)
        openapi.aspirate_in_place(flow_rate = flow_rate, volume = vol)
        # ---------------------------------WHERE TO DISPENSE---------------------------------------
        # x_d, y_d = converted_contour_centers[idx]
        # openapi.move_to_coordinates((x_d,y_d, dish_bottom + 2.5), min_z_height=1, verbose=False)
        # time.sleep(0.5)
        # openapi.dispense_in_place(flow_rate = flow_rate, volume = vol)
        # time.sleep(0.5)
        openapi.dispense(openapi.labware_dct['6'], well_mapping[idx], well_location='bottom', offset = (0.9,-0.9,0), volume = vol, flow_rate = flow_rate)
        time.sleep(0.3)
        openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=dish_bottom, verbose=False)
        idx += 1
        cuboid_chosen = False
        time.sleep(0.1)

    elif key_pressed == ord('d'):
        r = openapi.dispense_in_place(flow_rate = flow_rate, volume = vol)


cv2.destroyAllWindows()
keyboard.unhook_all()

21
Time taken: 11.79 seconds
20
Time taken: 11.74 seconds
19
Time taken: 11.84 seconds
17
Time taken: 11.69 seconds
23
Time taken: 11.74 seconds
18
Time taken: 11.67 seconds
19
Time taken: 11.71 seconds
15
Time taken: 11.87 seconds
15
Time taken: 11.85 seconds
13
Time taken: 11.92 seconds
13
Time taken: 11.86 seconds
11
Time taken: 11.92 seconds


In [65]:
np.sum(times) /24 * 384 / 60

32.32129096984863

In [17]:
openapi.control_run('stop') # Stop the robot

<Response [201]>

In [53]:
x, y = chosen.iloc[i][['cX', 'cY']].values

In [42]:
keyboard.unhook_all()

In [89]:
openapi.blow_out_in_place(50)

<Response [201]>

In [16]:
openapi.move_to_well(openapi.labware_dct['6'], 'A1', well_location='top', offset=(0,0,1), verbose=False)

<Response [201]>

In [91]:
r = openapi.dispense(openapi.labware_dct['3'], "A1", well_location='bottom', volume = 100, flow_rate = 200)

In [90]:
r = openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='bottom', offset = (0,0,0), volume = 100, flow_rate = 200)

In [ ]:
r = openapi.dispense(openapi.labware_dct['6'], well_mapping[idx], well_location='bottom', volume = vol, flow_rate = flow_rate)

In [88]:
json.loads(r.text)

{'data': {'id': 'c880966c-43ac-43d9-b0d4-2ee34b3a81bb',
  'createdAt': '2025-01-26T12:10:58.855084+00:00',
  'commandType': 'dispense',
  'key': 'c880966c-43ac-43d9-b0d4-2ee34b3a81bb',
  'status': 'failed',
  'params': {'labwareId': '492ea99c-5b92-4999-816b-6cda6d1c1d6f',
   'wellName': 'A1',
   'wellLocation': {'origin': 'bottom',
    'offset': {'x': 0, 'y': 0, 'z': 0},
    'volumeOffset': 0.0},
   'flowRate': 100.0,
   'volume': 1.0,
   'pipetteId': '6a51cb8a-aa10-45a7-bb93-666fb4664a70',
   'pushOut': 30.0},
  'error': {'id': '75a499ed-c890-4f81-a634-67511d459c77',
   'createdAt': '2025-01-26T12:10:58.892388+00:00',
   'isDefined': False,
   'errorType': 'CommandPreconditionViolated',
   'errorCode': '4004',
   'detail': 'Cannot push_out on a dispense that does not leave the pipette empty',
   'errorInfo': {'command': 'dispense', 'remaining-volume': '9.0'},
   'wrappedErrors': []},
  'startedAt': '2025-01-26T12:10:58.857117+00:00',
  'completedAt': '2025-01-26T12:10:58.892388+00:00'

# Picking procedure (Automatic)

In [18]:
def create_well_mapping(plate_type='384'):
    if plate_type == '384':
        rows = list("ABCDEFGHIJKLMNOP")
        columns = list(range(1, 25))
    elif plate_type == '96':
        rows = list("ABCDEFGH")
        columns = list(range(1, 13))
    else:
        raise ValueError("Unsupported plate type. Use '384' or '96'.")

    well_mapping = {}
    for i in range(len(rows) * len(columns)):
        row = rows[i // len(columns)]
        column = columns[i % len(columns)]
        well_mapping[i] = f"{row}{column}"

    return well_mapping

well_mapping = create_well_mapping('384')  # Change to '96' if using a 96 well plate

In [ ]:
# Flag to stop threads gracefully
stop_event = threading.Event()
pause_event = threading.Event()
pause_event.set()  # Start in paused state
coord_queue = queue.Queue()
cr = core.Core()

# ----------------------Robot configs-----------------------
calibration_data = utils.load_calibration_config(calibration_profile)
tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])
size_conversion_ratio = calibration_data['size_conversion_ratio']
one_d_ratio = calibration_data['one_d_ratio']

# ----------------------Picking configs-----------------------
vol = 10
dish_bottom = 10.3 #10.60 for 300ul, 9.5 for 200ul
pickup_offset = 0.5
pickup_height = dish_bottom + pickup_offset
flow_rate = 50
cuboid_size_theshold = (300, 500)
failure_threshold = 0.5
minimum_distance = 1.7

columns = list(range(1,25))
# rows = ['G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
# rows = ['A', 'B', 'C', 'D']
rows = ['C']
well_offset_x = 0.9 #384 well plate
well_offset_y = -0.9 #384 well plate

wells_to_fill = [f"{row}{column}" for row in rows for column in columns]
reversed_well_mapping = {v: k for k, v in well_mapping.items()}
wells_to_fill_indices = [reversed_well_mapping[well] for well in wells_to_fill]
idx = wells_to_fill_indices[0]
end_idx = wells_to_fill_indices[-1] + 1

# ----------------------Video configs-----------------------
circle_center = (int(1296.0), int(972.0))
circle_radius = 900

# ----------------------Dict for misses---------------------
misses = {}


class SharedSettings:
    def __init__(self):
        self.lock = threading.Lock()
        self.cuboid_chosen = False  # Movement speed (modifiable)
        self.idx = idx  # Current index (modifiable)
        self.local_timer_set = False

settings = SharedSettings()

def video_stream(): # Open default camera (change index if needed)
    window = cap.get_window()
    cuboid_choice = None

    start_time = time.time()
    local_timer_start = None

    while not stop_event.is_set():
        frame = cap.get_frame(undist=True)

        with settings.lock:
            cuboid_chosen = settings.cuboid_chosen
            local_timer_set = settings.local_timer_set
            idx = settings.idx

        if not pause_event.is_set() and not local_timer_set:
            local_timer_start = time.time()
            with settings.lock:
                settings.local_timer_set = True
        elif pause_event.is_set() and local_timer_set:
            local_timer_end = time.time()
            local_timer_duration = local_timer_end - local_timer_start
            print(f"Operation between pauses {local_timer_duration:.2f}")
            with settings.lock:
                settings.local_timer_set = False

        if idx >= end_idx:
            stop_event.set()
            break
        #-----------------------------------------VISION PROCESSING-----------------------------------------
        x, y, z = openapi.get_position(verbose=False)[0].values()
        (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 110), (0, 0, 0), -1)
        cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Filling well: {well_mapping[idx]}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        if pause_event.is_set():
            cv2.putText(frame, "Paused", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)



        cv2.circle(frame, circle_center, circle_radius + int(minimum_distance / one_d_ratio), (0, 0, 255), 2)
        mask = np.zeros_like(frame, dtype=np.uint8)
        cv2.circle(mask, circle_center, circle_radius + int(minimum_distance / one_d_ratio), (255, 255, 255), -1)
        masked_frame = cv2.bitwise_and(frame, mask)
        cv2.circle(frame, circle_center, circle_radius, (0, 255, 0), 2)

        if not cuboid_chosen:
            gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (11, 11), 0)
            thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,25,2) 
            kernel = np.ones((3,3),np.uint8)
            thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
            mask_inv = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
            thresh = cv2.bitwise_and(thresh, mask_inv)

            # Find contours in the masked frame
            contours, hei = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cr.cuboids = contours
            filtered_contours = [contour for contour in cr.cuboids if 30 < cv2.contourArea(contour) < 1000]
            cr.cuboid_dataframe(filtered_contours)

            cuboid_size_micron2 = cr.cuboid_df.area * size_conversion_ratio * 1000000
            cuboid_diameter = 2 * np.sqrt(cuboid_size_micron2 / np.pi)
            dist_mm = cr.cuboid_df.min_dist * one_d_ratio
            cr.cuboid_df['diameter_microns'] = cuboid_diameter
            cr.cuboid_df['min_dist_mm'] = dist_mm
            
            # Filter out elongated contours

            pickable_cuboids = cr.cuboid_df.loc[(cuboid_size_theshold[0] < cr.cuboid_df['diameter_microns']) & 
                                                (cr.cuboid_df['diameter_microns'] < cuboid_size_theshold[1]) &
                                                ((cr.cuboid_df['aspect_ratio'] > 0.75) | (cr.cuboid_df['aspect_ratio'] < 1.25)) &
                                                (cr.cuboid_df['circularity'] > 0.6)].copy()

            # Check if cuboid centers are within the circle radius from the current circle center
            pickable_cuboids['distance_to_center'] = pickable_cuboids.apply(
                lambda row: np.sqrt((row['cX'] - circle_center[0])**2 + (row['cY'] - circle_center[1])**2), axis=1
            )
            pickable_cuboids = pickable_cuboids[pickable_cuboids['distance_to_center'] <= circle_radius]
            isolated = pickable_cuboids.loc[pickable_cuboids.min_dist_mm > minimum_distance]
            draw = isolated.contour.values.tolist()
            cv2.drawContours(frame, filtered_contours, -1, (0, 0, 255), 2)
            cv2.drawContours(frame, pickable_cuboids.contour.values.tolist(), -1, (0, 255, 255), 2)
            cv2.drawContours(frame, draw, -1, (0, 255, 0), 2)

        #-----------------------------------------------INTERACTION------------------------------------------------

            # if not cuboid_chosen and len(isolated) > 0:
            #     if cuboid_choice is not None:
            #         prev_x, prev_y = cuboid_choice[['cX', 'cY']].values[0]
                    
            #         cv2.circle(frame, (int(prev_x), int(prev_y)), int(round(failure_threshold / one_d_ratio)), (255, 0, 0), 2)
            #         distances = cr.cuboid_df.apply(lambda row: np.sqrt((row['cX'] - prev_x)**2 + (row['cY'] - prev_y)**2), axis=1).to_numpy()
            #         distances *= one_d_ratio
            #         if any(distances <= failure_threshold):
            #             print("Miss detected ...")
            #             with settings.lock:
            #                 settings.idx -= 1

            #     cuboid_choice = isolated.sample(n=1) 
            #     cv2.drawContours(frame, cuboid_choice.contour.values.tolist(), -1, (255, 0, 0), 2)

        if not coord_queue.full() and not cuboid_chosen and not pause_event.is_set() and len(isolated) > 0:
            if cuboid_choice is not None:
                prev_x, prev_y = cuboid_choice[['cX', 'cY']].values[0]
                
                cv2.circle(frame, (int(prev_x), int(prev_y)), int(round(failure_threshold / one_d_ratio)), (255, 0, 0), 2)
                distances = cr.cuboid_df.apply(lambda row: np.sqrt((row['cX'] - prev_x)**2 + (row['cY'] - prev_y)**2), axis=1).to_numpy()
                distances *= one_d_ratio
                if any(distances <= failure_threshold):
                    with settings.lock:
                        settings.idx -= 1
                    print(f"Miss detected at well {well_mapping[idx]}.")
                    if well_mapping[idx] in misses:
                        misses[well_mapping[idx]] += 1
                    else:
                        misses[well_mapping[idx]] = 1

            cuboid_choice = isolated.sample(n=1) 
            cv2.drawContours(frame, cuboid_choice.contour.values.tolist(), -1, (255, 0, 0), 2)
            # cv2.imwrite(str(paths.BASE_DIR)+'\\outputs\\images\\'+f"frame_{idx}.png", frame)

            cX, cY = cuboid_choice[['cX', 'cY']].values[0]
            X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)
            x, y, _ = openapi.get_position(verbose=False)[0].values()
            diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
            X = X_init + diff[0] + offset[0]
            Y = Y_init + diff[1] + offset[1]

            coord_queue.put((X, Y))
            with settings.lock:
                settings.cuboid_chosen = True
        elif len(isolated) == 0 and not pause_event.is_set():
            pause_event.set()
            print("No cuboids found in the selected region. Pausing...")

        cv2.imshow(cap.window_name, frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            stop_event.set()
        elif key == ord('p'):
            if pause_event.is_set():
                pause_event.clear()
                print("Resuming movement...")
            else:
                print("Pausing movement...")
                pause_event.set()

    cv2.destroyAllWindows()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    if local_timer_start:
        print(f"Last local time: {end_time - local_timer_start} seconds")
    # print(f"Last index: {well_mapping[idx]}")

def robot_movement():
    openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=dish_bottom, verbose=False)
    while not stop_event.is_set():
        if pause_event.is_set():
            time.sleep(0.1)  # Small sleep to prevent excessive CPU usage
            continue  # Skip to next iteration while paused
        # print("Moving robot...")
        try:
            # Get latest coordinates from the queue (non-blocking)
            x, y = coord_queue.get(timeout=1)  # Timeout prevents indefinite blocking
            openapi.move_to_coordinates((x, y, pickup_height+20), min_z_height=dish_bottom, verbose=False, force_direct=True)
            openapi.move_to_coordinates((x, y, pickup_height), min_z_height=dish_bottom, verbose=False, force_direct=True)
            openapi.aspirate_in_place(flow_rate = flow_rate, volume = vol)
            openapi.move_relative('z', 20)

            with settings.lock:
                idx = settings.idx

            openapi.move_to_well(openapi.labware_dct['6'], well_mapping[idx], well_location='top', offset=(well_offset_x,well_offset_y,5), verbose=False, force_direct=True)
            openapi.dispense(openapi.labware_dct['6'], well_mapping[idx], well_location='bottom', offset=(well_offset_x, well_offset_y, 0), volume = vol, flow_rate = flow_rate)
            time.sleep(0.3)
            openapi.move_to_well(openapi.labware_dct['6'], well_mapping[idx], well_location='top', offset=(well_offset_x,well_offset_y,5), verbose=False)
            openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=dish_bottom, verbose=False, force_direct=True)
            time.sleep(0.5)
            with settings.lock:
                settings.cuboid_chosen = False
                settings.idx += 1 
        except queue.Empty:
            pass  # No new coordinates, continue looping
# Create threads
video_thread = threading.Thread(target=video_stream, daemon=True)
robot_thread = threading.Thread(target=robot_movement, daemon=True)

# Start threads
video_thread.start()
robot_thread.start()

# Wait for threads to finish
video_thread.join()
robot_thread.join()

Resuming movement...
Miss detected at well C9.
Miss detected at well C9.
Miss detected at well C10.
Miss detected at well C14.
Miss detected at well C17.
Miss detected at well C21.
Miss detected at well C21.
Elapsed time: 202.74 seconds
Last local time: 191.83526730537415 seconds


In [34]:
misses

{'C9': 2, 'C10': 1, 'C14': 1, 'C17': 1, 'C21': 2}

In [45]:
idx = 0
while idx < len(misses):
    well_name, miss_count = list(misses.items())[idx]
    r = openapi.aspirate(openapi.labware_dct['6'], f"{well_name}", well_location = 'bottom', offset = (well_offset_x,well_offset_y,0), volume = miss_count*vol, flow_rate = 5)
    responce_dict = json.loads(r.text)['data']
    if responce_dict['status'] == 'failed':
        if responce_dict['error']['errorType'] == 'InvalidAspirateVolumeError':
            print('Dumping fluid')
            openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
    else:
        idx += 1
    
openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
openapi.move_relative('z', 20)

<Response [201]>

In [21]:
openapi.dispense(openapi.labware_dct['3'], "A1", well_location='center', offset = (0,0,0), volume = 10, flow_rate = 50)

<Response [201]>

In [7]:
test = queue.Queue()

In [28]:
test.put((3,4))

In [22]:
one, two = test.get(timeout=1)

In [29]:
test.full()

False

# Picking procedure (Auto + class)

# Cuboid recognition

In [ ]:
calibration_data = utils.load_calibration_config(calibration_profile)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])


window = cap.get_window()

def on_mouse_click(event, x, y, flags, param):
    global circle_center
    global circle_radius
    global filtered_contours
    global X_init, Y_init

    if event == cv2.EVENT_MBUTTONDOWN:
        circle_center = (x, y)

    if event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:
            circle_radius += 10
        else:
            circle_radius -= 10

    # if event == cv2.EVENT_LBUTTONDBLCLK:
    #     for contour in filtered_contours:
    #         r=cv2.pointPolygonTest(contour, (x,y), False)
    #         if r>0:
    #             M = cv2.moments(contour)
    #             if M["m00"] != 0:
    #                 cX = int(M["m10"] / M["m00"])
    #                 cY = int(M["m01"] / M["m00"])
    #                 X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)

    #                 x, y, _ = openapi.get_position(verbose=False)[0].values()
    #                 diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
    #                 X = X_init + diff[0] + offset[0]
    #                 Y = Y_init + diff[1] + offset[1]
                    
    #                 print(f"Robot coords: ({x}, {y})")
    #                 print(f"Clicked on: ({X}, {Y})")
    #                 openapi.move_to_coordinates((X, Y, 15), min_z_height=1, verbose=False)
    #                 # openapi.aspirate_in_place(flow_rate = 75, volume = 10)

                    
    #             else:
    #                 print("Contour center could not be found")

    # if event == cv2.EVENT_RBUTTONDOWN:
    #     x, y, _ = openapi.get_position(verbose=False)[0].values()
    #     # openapi.move_to_coordinates((x, y, 100), min_z_height=1)
    #     openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=1, verbose=False)

        

cv2.setMouseCallback(cap.window_name, on_mouse_click)
circle_center = (int(1296.0), int(972.0))
circle_radius = 300
# manual_movement = utils.ManualRobotMovement(openapi)

while True:
    frame = cap.get_frame(undist=True)
    # x, y, z = openapi.get_position(verbose=False)[0].values()
    # (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    # cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    # cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    # cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    cv2.circle(frame, circle_center, circle_radius, (255, 0, 0), 2)
    # Create a mask with the same dimensions as the frame
    mask = np.zeros_like(frame, dtype=np.uint8)

    # Draw a filled circle on the mask
    cv2.circle(mask, circle_center, circle_radius, (255, 255, 255), -1)

    # Apply the mask to the frame
    masked_frame = cv2.bitwise_and(frame, mask)

    # Convert the masked frame to grayscale
    gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)
    # Apply thresholding to the grayscale image
    # _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    gray = cv2.GaussianBlur(gray, (11, 11), 0)

    thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,25,2) 
    kernel = np.ones((3,3),np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    # Fill the area outside the circle with black pixels
    # Convert the mask to grayscale
    mask_inv = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    thresh = cv2.bitwise_and(thresh, mask_inv)

    # frame[thresh == 255] = [0, 255, 0]

    # Find contours in the masked frame
    contours, hei = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)



    # Filter the contours to exclude the outermost
    filtered_contours = [contour for contour, h in zip(contours, hei[0]) if h[3] == 1]
    # Filter the contours by size
    filtered_contours = [contour for contour in contours if 15 < cv2.contourArea(contour) < 1000]
    # Draw the contours on the frame
    cv2.drawContours(frame, filtered_contours, -1, (0, 255, 0), 2)

    cv2.imshow(cap.window_name, frame)
    key_pressed = cv2.waitKey(1)

    if key_pressed == ord('q'):
        keyboard.unhook_all()
        break

        
cv2.destroyAllWindows()

In [70]:
openapi.move_to_well(openapi.labware_dct['6'], 'A1', well_location='top', offset=(1,-1,0), verbose=False)

<Response [201]>